In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        nochild.update({key : filnary})

In [3]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [4]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process(Thing):
        namespace = onto
    class molecular_function(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [5]:
#classmaker function to make FTC classes

def classmaker(term, parentclass):
                classnameA = 'GOA' + str(term)
                classnameP = 'GOP' + str(term)
                with onto:
                    classname = types.new_class(classnameA, (parentclass,))
                    classname = types.new_class(classnameP, (parentclass,))
                return 0

In [6]:
#remove anything that's not a biological process or molecular function from kilnary

c = 0
keys = []
for key, value in kilnary.items():
    if value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
print(c)
for i in keys:
    del kilnary[i]

1522


In [7]:
#Using classmaker to make FTC classes

for key, value in kilnary.items():
    if value['aspect'] == 'biological_process':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'negatively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
    if value['aspect'] == 'molecular_function':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'negatively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)


In [8]:
#number of child classes linked by 'regulates'

print(len(kilnary.items()), len(nochild.items()), len(list(onto.classes())))
c=0
l = []

for key, value in kilnary.items():
    for item in value['children']:
        if item['relation'] == 'positively_regulates' or item['relation'] == 'negatively_regulates' or item['relation'] == 'regulates': 
            if value['aspect'] == 'biological_process' or value['aspect'] == 'molecular_function':
                c+=1

print(c)        

20749 23421 20250
11161


In [9]:
#make bp and mf classes

for key, value in nary.items():

    if value.get('aspect') != None and value['aspect'] == 'biological_process':
        with onto:
            classname = types.new_class(key, (biological_process,))
    if value.get('aspect') != None and value['aspect'] == 'molecular_function':
        with onto:
            classname = types.new_class(key, (molecular_function,))

In [8]:
#assign appropriate labels to classes

x = list(onto.classes())
for y in x:
    term = 'GO:' + str(y)[8:]
    if str(y)[7] == ':':
        y.label = nary[str(y)[5:]]['name']
    if str(y)[7] == 'A':
        y.label = 'anti_' + nary[term]['name'] + '_agent'
    
    if str(y)[7] == 'P':
        y.label = 'pro_' + nary[term]['name'] + '_agent'

In [9]:
#list of ids from the created classes in the two text formats

ids = []
ontoids = []
for cl in list(onto.classes()):
    cl = str(cl)
    ontoids.append(cl)
    clt = cl[5:7] +':' + cl[8:]
    ids.append(clt)

In [10]:
#propertymaker function for making properties

def propertymaker(pname, pdomain, prange):
    with onto:
            classname = types.new_class((pname), (ObjectProperty,), )
    return 0

In [11]:
#parsing biological process and molecular function classes

reldict = {}


for key, value in kilnary.items():            
    for item in value['children']:
        if reldict.get(item['relation']) == None:
            reldict[item['relation']] = [[],[]]
            propertymaker(item['relation'], 'void', 'void')
    for item in value['children']:
        if item['id'] in ids:
            reldict[item['relation']][1].append(ids[ids.index(value['id'])])
            reldict[item['relation']][0].append(ids[ids.index(item['id'])])
        

ValueError: 'GO:0000012' is not in list

In [14]:
len(list(kilnary.keys()))

20749

In [12]:
#listing positively_regulates and negatively_regulates as subproperties

with onto:
    class positively_regulates(onto.regulates):
        pass
    class negatively_regulates(onto.regulates):
        pass

In [13]:
#converting string to class object in reldict lists

for key, value in reldict.items():
    for item in value[0]:
        value[0][value[0].index(item)] = getattr(onto, item)
    for item in value[1]:
        value[1][value[1].index(item)] = getattr(onto, item)


In [14]:
#setting relations

for key, value in ded.items():
    setattr(key, str(value[0])[2:-2], value[1])

NameError: name 'ded' is not defined

In [15]:
#setting relations

for key, value in reldict.items():
    for item in value[0]:
        setattr(item, key, [value[1][value[0].index(item)]])

In [16]:
onto.save(file = "testowl3", format = "rdfxml")